- Aim: Combining OCR & LLM search to get information from a scanned pdf.
- This notebook tests AWS Textract on a few images (pdf converted to images)

In [49]:
import io
from PIL import Image, ImageDraw
from dotenv import load_dotenv; load_dotenv()
import json
import os
import sys
import boto3
import botocore

In [50]:
# Connect to Amazon Textract to detect text in the document
client = boto3.client("textract", region_name="eu-west-2", aws_access_key_id='', 
aws_secret_access_key='')

In [51]:
def analyze_file(
        client, feature_types, QueriesConfig, *, document_file_name=None, document_bytes=None
    ):
        """
        Detects text and additional elements, such as forms or tables, in a local image
        file or from in-memory byte data.
        The image must be in PNG or JPG format.

        :param feature_types: The types of additional document features to detect.
        :param document_file_name: The name of a document image file.
        :param document_bytes: In-memory byte data of a document image.
        :return: The response from Amazon Textract, including a list of blocks
                 that describe elements detected in the image.
        """
        
        with open(document_file_name, "rb") as document_file:
            document_bytes = document_file.read()
        response = client.analyze_document(
            Document={"Bytes": document_bytes}, FeatureTypes=feature_types, #QueriesConfig=QueriesConfig
        )

        return response

                  

In [52]:
response = analyze_file(client, document_file_name='test.png', feature_types=['TABLES'], QueriesConfig={
            "Queries": [{
                "Text": "What is the vehicle registration number?",
                "Alias": "VID"
            },
            {
                "Text": "What is the GTW value?",
                "Alias": "GTW"
            },
            {
                "Text": "What is the GVW value?",
                "Alias": "GVW"
            },
            {
                "Text": "What is the overall test result?",
                "Alias": "test_result"
            }
            ]
        })

In [53]:
response

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 1.0,
     'Height': 1.0,
     'Left': 0.0,
     'Top': 0.0},
    'Polygon': [{'X': 0.0, 'Y': 3.2322341212420724e-06},
     {'X': 0.9977295994758606, 'Y': 0.0},
     {'X': 1.0, 'Y': 1.0},
     {'X': 0.001291565247811377, 'Y': 1.0}]},
   'Id': '450cfdab-679f-4763-a166-994608aba74d',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['05ae0d60-dac4-45ee-ab31-44aae0789e5a',
      'e8ed52a3-d5d9-4ebe-a833-06c82b070a79',
      '71947948-d832-4156-8096-1a6531c0e182',
      '370eca1d-5432-450d-bbc4-f9a1ad159a46',
      '44a850d8-a59e-43aa-98f7-3c01a740ea0a',
      '5f56a3c4-ae4c-44bd-9be0-8ba107a2c01a',
      '5ddb578c-a3ec-4bdf-9cf2-5a4e1bc64c43',
      '126491e4-99ec-47a1-862e-2b8a0baa5152',
      '1874b5e9-857a-41ea-b6a3-a67ae79c7cd3',
      '2ac42f26-9deb-44f3-ac90-89fae56c365d',
      '02395055-3d22-4287-bdb1-c0d6c8dfeed9',
      '18b2292f-ed41-4266-8b85-199be6b36c37',
      '006

In [54]:
response['Blocks'][1]['Text']

'DTP Number : 6174L'

In [55]:
response['Blocks'][1].keys()

dict_keys(['BlockType', 'Confidence', 'Text', 'Geometry', 'Id', 'Relationships'])

In [56]:
brake_text = ""
for i in range(0, len(response['Blocks'])):
    if response['Blocks'][i]['BlockType'] == 'LINE':
        if 'Text' in response['Blocks'][i].keys():
            brake_text = brake_text + response['Blocks'][i]['Text'] + " "
        brake_text = brake_text + "\n"

print(brake_text)

DTP Number : 6174L 
GTW : 50000kg 
Vehicle Make : SCANIA 
GVN 27000kg 
Vehicle Type : 3 AXLE RIGID HGV 
Vehicle Reg 
: 
MAX. FORCE 
AXLE 1 - 6358kg 
AG 
CUALITY 
IMBALANC 
1762kg (L) 
cass 
1762kg 
N/S 
3224kg 
Pass 
Pass 
Service 
Sase 
2011kg 
(12%) 
2011kg (L) 
o/s 
3134kg 
Pass 
Pass 
1876kg 
1876kg 
-- 
-- 
Secondary N/S 
3224kg 
-- 
-- 
1693kg ( 9%) 
1693kg 
-- 
-- 
o/s 
3134kg 
-- 
2149kg 
2149kg (L) 
3224kg 
-- 
N/S 
-- 
Parking 
2018kg 
( 6%) 
2018kg 
-- 
o/s 
3134kg 
-- 
-- 
AXLE 2 - 3013kg 
BIND 
TIME LAG 
IMBALANCE 
MAX. FORCE 
WEIGHT 
OVALITY 
N/S 
Pass 
-- 
1443kg 
-- 
929kg 
929kg (L) 
Service 
-- 
o/s 
1570kg 
1163kg (20%) 
1163kg (L) 
Pass 
-- 
-- 
Secondary N/S 
1443kg 
-- 
-- 
889kg 
889kg (L) 
-- 
-- 
-- 
1197kg 
(25%) 
1197kg (L) 
O/S 
1570kg 
914kg (L) 
Parking 
N/S 
1443kg 
-- 
-- 
-- 
914kg 
O/S 
1570kg 
-- 
-- 
-- 
1101kg 
(16%) 
1101kg (L) 
AXLE 3 - 2128kg 
WEIGHT 
BIND 
TIME LAG 
OVALITY 
IMBALANCE 
MAX. FORCE 
Service 
N/S 
1044kg 
Pass 
-- 
-- 
673kg 
673kg

In [57]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import math
from IPython.core.display import HTML

In [58]:
"""Helper utilities for working with Amazon Bedrock from Python notebooks"""
# Python Built-Ins:
from typing import Optional

# External Dependencies:
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [59]:
os.environ["AWS_DEFAULT_REGION"] = "eu-central-1"

bedrock_runtime = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

Create new client
  Using region: eu-central-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.eu-central-1.amazonaws.com)


In [60]:
"""General helper utilities the workshop notebooks"""
# Python Built-Ins:
from io import StringIO
import textwrap


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

In [61]:
# If you'd like to try your own prompt, edit this parameter!
customer_input = f"""Below is a summary of one break test for a single vehicle. 
Based on the break test, get the vehicle registration number, GVW, GTW and test result. 
The information needs to be on the same line to be connected.
Format the answer as a Json. If any of the values are unknown, put NaN as the answer.
For example: ["vehicle_registration_number": "RV101010", "GVW": "1000kg", "GTW": "NaN", "test_result": "Fail"]
Summary of the break test: [{brake_text}]"""

In [62]:
print(customer_input)

Below is a summary of one break test for a single vehicle. 
Based on the break test, get the vehicle registration number, GVW, GTW and test result. 
The information needs to be on the same line to be connected.
Format the answer as a Json. If any of the values are unknown, put NaN as the answer.
For example: ["vehicle_registration_number": "RV101010", "GVW": "1000kg", "GTW": "NaN", "test_result": "Fail"]
Summary of the break test: [DTP Number : 6174L 
GTW : 50000kg 
Vehicle Make : SCANIA 
GVN 27000kg 
Vehicle Type : 3 AXLE RIGID HGV 
Vehicle Reg 
: 
MAX. FORCE 
AXLE 1 - 6358kg 
AG 
CUALITY 
IMBALANC 
1762kg (L) 
cass 
1762kg 
N/S 
3224kg 
Pass 
Pass 
Service 
Sase 
2011kg 
(12%) 
2011kg (L) 
o/s 
3134kg 
Pass 
Pass 
1876kg 
1876kg 
-- 
-- 
Secondary N/S 
3224kg 
-- 
-- 
1693kg ( 9%) 
1693kg 
-- 
-- 
o/s 
3134kg 
-- 
2149kg 
2149kg (L) 
3224kg 
-- 
N/S 
-- 
Parking 
2018kg 
( 6%) 
2018kg 
-- 
o/s 
3134kg 
-- 
-- 
AXLE 2 - 3013kg 
BIND 
TIME LAG 
IMBALANCE 
MAX. FORCE 
WEIGHT 
OVALITY 
N

In [63]:
# first fetch possible styles and give options to customer
prompt = """

Human: 
""" + customer_input + """

Assistant:"""
# body = json.dumps({"inputText": prompt, "max_tokens_to_sample": 500})
body = json.dumps({"inputText":prompt,"textGenerationConfig":{"maxTokenCount":1000,"stopSequences":[],"temperature":0,"topP":1}})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
claudeResponse = ""

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
print(response_body["results"][0]["outputText"])


```tabular-data-json
{
    "rows": [
        {
            "Vehicle Registration Number": "RV101010",
            "GVW": "1000kg",
            "GTW": "50000kg",
            "Test Result": "Pass"
        }
    ]
}
```
